In [1]:
# | default_exp agent.introspector

In [2]:
# | export 
from typing import Dict, Any, Optional
from rich.console import Console


class IntrospectAgent:
    """LLM-powered task execution evaluation agent"""
    
    def __init__(self, agent, console: Console):
        self.agent = agent
        self.console = console
    
    def evaluate_execution(
        self, 
        task_context: Dict[str, Any],
        execution_result: str,
        success_criteria: str,
        expected_outputs: list = None
    ) -> Dict[str, Any]:
        """Evaluate task execution quality and provide feedback"""
        
        prompt = f"""
TASK EXECUTION EVALUATION

TASK CONTEXT:
- Task ID: {task_context.get('task_id', 'unknown')}
- Description: {task_context.get('description', '')}
- Purpose: {task_context.get('purpose', '')}

EXECUTION DETAILS:
- Result: {execution_result}
- Success Criteria: {success_criteria}
- Expected Outputs: {expected_outputs or []}

EVALUATION FRAMEWORK:
Analyze the execution across these dimensions:

1. COMPLETENESS (0-10): Did it fulfill all requirements and create expected outputs?
2. QUALITY (0-10): Code quality, documentation, best practices followed?
3. CORRECTNESS (0-10): Does the solution work as intended?
4. EFFICIENCY (0-10): Is the approach optimal and well-structured?

SCORING GUIDELINES:
- 9-10: Excellent, exceeds expectations
- 7-8: Good, meets requirements with minor issues
- 5-6: Acceptable but needs improvement
- 3-4: Poor, significant issues
- 0-2: Failed, major problems

RESPONSE FORMAT (JSON):
{{
  "overall_score": <0-10>,
  "category_scores": {{
    "completeness": <0-10>,
    "quality": <0-10>, 
    "correctness": <0-10>,
    "efficiency": <0-10>
  }},
  "success": <true/false>,
  "issues_found": ["issue1", "issue2"],
  "recommendations": ["recommendation1", "recommendation2"],
  "feedback_for_retry": "Specific guidance for improvement",
  "next_action": "proceed|retry|major_revision",
  "reasoning": "Detailed explanation of the evaluation"
}}

Provide thorough, constructive evaluation focusing on actionable feedback.
"""

        try:
            response = self.agent.run(prompt, stream=True, max_iterations=1)
            content = response.get("content", "")
            
            # Try to parse JSON response
            import json
            try:
                result = json.loads(content)
                # Ensure required fields exist
                result.setdefault("success", result.get("overall_score", 0) >= 7)
                result.setdefault("performance_score", result.get("overall_score", 0))
                return result
            except json.JSONDecodeError:
                # Fallback if JSON parsing fails
                return {
                    "overall_score": 5.0,
                    "success": False,
                    "feedback_for_retry": "Evaluation parsing failed - please retry",
                    "next_action": "retry",
                    "reasoning": f"Raw response: {content}"
                }
                
        except Exception as e:
            self.console.print(f"❌ Introspection error: {e}")
            return {
                "overall_score": 0.0,
                "success": False,
                "feedback_for_retry": f"Introspection failed: {str(e)}",
                "next_action": "retry",
                "reasoning": "Agent evaluation failed"
            }


def create_introspection(
    task_context: Dict[str, Any],
    execution_result: str, 
    success_criteria: str,
    expected_outputs: list = None,
    agent = None,
    console: Console = None
) -> Dict[str, Any]:
    """Convenience function to create and run introspection"""
    
    if not console:
        console = Console()
    
    introspect_agent = IntrospectAgent(agent, console)
    return introspect_agent.evaluate_execution(
        task_context, execution_result, success_criteria, expected_outputs
    )
